In [30]:
import apache_beam as beam
import google
from apache_beam.options.pipeline_options import PipelineOptions
import sqlalchemy as sqlalchemy
from google.cloud.sql.connector import Connector
import logging
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.io import ReadFromCsv,WriteToText
from apache_beam.io import WriteToCsv,WriteToText



# Enable logging
logging.basicConfig(level=logging.INFO)

class WriteToSQL(beam.DoFn):
    def setup(self):
        # function to return database connection
        def get_conn():
            connector = Connector()
            conn = connector.connect(
                "lithe-catbird-434312-p9:us-central1:assaydemo",  # Update with correct connection name
                "mysql",
                user="dan",
                password="Arapkdan@12",
                db="assay_demo"
            )
            return conn
        
        # create connection pool
        self.pool = sqlalchemy.create_engine(
            "mysql+pymysql://",  # Corrected driver
            creator=get_conn
        )
    
    def process(self, element):
        """Process each element and write to Cloud SQL"""
        # Assuming the element is a CSV line, we can parse it
        employee_id, company_id, department_id, person_id, employment_type, employment_status, designation_id = element.split(",")  # Adjust as per your actual schema
        
        # Insert data into the database
        with self.pool.connect() as connection:
            connection.execute(
                "INSERT INTO employees (employee_id, company_id, department_id, person_id, employment_type, employment_status, designation_id) "
                VALUES (%s, %s, %s, %s, %s, %s, %s)",
                (employee_id, company_id, department_id, person_id, employment_type, employment_status, designation_id)
            )
            logging.info(f"Inserted record {employee_id}")

        # Yield a success message or return an empty list
        #yield f"Inserted record {employee_id}"  # Yielding a message for tracking
    def teardown(self):
        # Close the connection pool
        self.pool.dispose()


# Define the pipeline options
options = PipelineOptions()
                            

with beam.Pipeline() as p :
    # read the csv file 
    read_csv =  p | 'Read' >> beam.io.ReadFromText('gs://assay_demo1/bsc_appraisals.csv')| 'Print'  >> beam.Map(print)


    
p.run()

1,2288,120947
2,2288,120947
3,2288,120947
4,4707,88653
5,2288,132841
6,4707,88653
1,2288,120947
2,2288,120947
3,2288,120947
4,4707,88653
5,2288,132841
6,4707,88653
